# Segmentação de Ovos com Processamento Digital de Imagens
Este notebook realiza a segmentação automática de ovos com base no artigo:
**“Implementação de algoritmo de processamento digital de imagens para segmentação de ovos de galinha”**

**Bibliotecas utilizadas:** OpenCV, NumPy e Matplotlib
Cada etapa será documentada com base nas práticas do artigo e nas documentações oficiais das bibliotecas.

In [ ]:
!pip install opencv-python-headless numpy matplotlib


## Etapa 1: Importação das bibliotecas
- `cv2`: biblioteca OpenCV para manipulação de imagens
- `numpy`: manipulação numérica
- `matplotlib`: visualização dos resultados
- `itertools.combinations`: para conectar componentes na visualização
- `os`: manipulação de diretórios

In [18]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from itertools import combinations
import os

## Etapa 2: Aquisição da imagem
A imagem é carregada a partir de um caminho local. O OpenCV utiliza `cv2.imread()` para ler arquivos de imagem.
Se a imagem não for encontrada, `None` será retornado.

In [ ]:
image_path = "ovos.png"  # Exemplo: 'dados/ovos_grade.jpg'
image = cv2.imread(image_path)

# Exibe a imagem original
plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Imagem Original - Aquisição")
plt.axis("off")
plt.show()

## Etapa 3: Pré-processamento
### 3.1 Conversão para escala de cinza
Transforma a imagem para tons de cinza com `cv2.cvtColor()`.
Segundo o artigo e a documentação do OpenCV, isso reduz a complexidade e é essencial para etapas como detecção de bordas.

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

plt.imshow(gray, cmap='gray')
plt.title("Imagem em Escala de Cinza")
plt.axis("off")
plt.show()

### 3.2 Suavização com Filtro Gaussiano
Aplicado com `cv2.GaussianBlur()` para reduzir ruídos e suavizar detalhes menores, facilitando a detecção de bordas.
Conforme o artigo, é uma etapa crítica antes do uso do algoritmo de Canny.

In [ ]:
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

plt.imshow(blurred, cmap='gray')
plt.title("Imagem Suavizada - Filtro Gaussiano")
plt.axis("off")
plt.show()

### 3.3 Detecção de Bordas com Canny
O algoritmo de Canny é utilizado para detectar bordas com base nos gradientes de intensidade.
A documentação do OpenCV recomenda essa técnica como robusta para detecção de contornos.

In [ ]:
edges = cv2.Canny(blurred, 30, 150)

plt.imshow(edges, cmap='gray')
plt.title("Detecção de Bordas - Canny")
plt.axis("off")
plt.show()

## Etapa 4: Extração de Contornos
Utiliza `cv2.findContours()` para extrair os contornos externos na imagem de bordas. Depois, filtra os contornos com área suficiente para evitar ruídos.

In [ ]:
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
rectangles = [cv2.boundingRect(c) for c in contours if cv2.contourArea(c) > 100]

image_copy = image.copy()
for (x, y, w, h) in rectangles:
    cv2.rectangle(image_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)

plt.imshow(cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB))
plt.title("Contornos Detectados")
plt.axis("off")
plt.show()


### Agrupamento de ovos por linha
Agrupa os ovos detectados em diferentes linhas com base na coordenada vertical dos centróides dos retângulos. Isso facilita a ordenação e nomeação dos arquivos.

In [56]:
def calculate_centroid(rect):
    x, y, w, h = rect
    return (x + w // 2, y + h // 2)

def search_components(start_rect, rectangles, visited, max_distance=20):
    connected = []
    stack = [start_rect]

    while stack:
        rect = stack.pop()
        if rect not in visited:
            connected.append(rect)
            visited.add(rect)
            cy1 = calculate_centroid(rect)[1]
            for other in rectangles:
                if other not in visited:
                    cy2 = calculate_centroid(other)[1]
                    if abs(cy1 - cy2) < max_distance:
                        stack.append(other)
    return connected

visited = set()
connected_components = []

for rect in rectangles:
    if rect not in visited:
        group = search_components(rect, rectangles, visited)
        connected_components.append(group)

# Ordena os grupos pela coordenada vertical
connected_components.sort(key=lambda comp: calculate_centroid(comp[0])[1])


### Salvamento das imagens segmentadas
Salva cada ovo segmentado como uma imagem individual na pasta `eggs_segmented`, nomeando os arquivos conforme sua posição.

In [57]:
os.makedirs("eggs_segmented", exist_ok=True)

for i, component in enumerate(connected_components):
    component.sort(key=lambda r: calculate_centroid(r)[0])  # Ordena pela horizontal
    for j, (x, y, w, h) in enumerate(component):
        crop = image[y:y+h, x:x+w]
        filename = f"eggs_segmented/row{i+1}_egg{j+1}.png"
        cv2.imwrite(filename, crop)


### Visualização final com linhas entre ovos da mesma fileira
Conecta os centróides dos ovos em cada linha com linhas tracejadas, ajudando a visualizar a distribuição espacial.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

for group in connected_components:
    for (x, y, w, h) in group:
        ax.add_patch(Rectangle((x, y), w, h, edgecolor='red', fill=False, linewidth=2))
    for r1, r2 in combinations(group, 2):
        c1 = calculate_centroid(r1)
        c2 = calculate_centroid(r2)
        ax.plot([c1[0], c2[0]], [c1[1], c2[1]], 'r--')

ax.set_title("Segmentação Final dos Ovos")
ax.axis("off")
plt.show()


### Numeração dos ovos na imagem
Adiciona numeração visual em cada ovo detectado, facilitando a contagem manual e a verificação dos resultados.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

count = 1
for group in connected_components:
    for (x, y, w, h) in group:
        ax.add_patch(Rectangle((x, y), w, h, edgecolor='red', fill=False, linewidth=2))
        cx, cy = x + w // 2, y + h // 2
        ax.text(cx, cy, str(count), color='yellow', fontsize=10, ha='center')
        count += 1

ax.set_title("Segmentação com Numeração dos Ovos")
ax.axis("off")
plt.show()